# EDA: Meteorological Data and SRAG Cases (2019)

Este notebook explora dados meteorológicos diários obtidos da API **NASA POWER** para algumas capitais brasileiras (São Paulo, Rio de Janeiro e Manaus) durante o ano de 2019 e relaciona essas informações com as contagens de casos de Síndrome Respiratória Aguda Grave (SRAG) registrados no Sistema SIVEP‑Gripe para os mesmos estados.

**Objetivos:**

- Carregar e analisar estatísticas descritivas da temperatura média diária (T2M) em 2019 para SP, RJ e AM.
- Cruzar os dados de temperatura com as contagens diárias de SRAG (2019) por estado.
- Investigar possíveis correlações entre temperatura e número de casos de SRAG.

As fontes de dados utilizadas são descritas no diretório `referencias` do repositório e no arquivo `origem_dados.md`.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Carregar dados meteorológicos (temperatura média diária)
weather = pd.read_csv('../data/NASA/temperature/weather_2019_states.csv')
weather['date'] = pd.to_datetime(weather['date'])

# Carregar dados de casos agregados (SRAG)
# A versão agregada foi baixada do repositório e salva localmente com este notebook.
sivep = pd.read_csv('../data/SIVEP/2019/aggregated_sivep_2019.csv')
sivep['DT_SIN_PRI'] = pd.to_datetime(sivep['DT_SIN_PRI'])

# Filtrar apenas os estados analisados (SP, RJ e AM)
sivep_sub = sivep[sivep['SG_UF'].isin(['SP','RJ','AM'])].copy()

# Renomear coluna para facilitar merge
sivep_sub = sivep_sub.rename(columns={'DT_SIN_PRI': 'date', 'SG_UF':'state', 'COUNT':'cases'})

# Mesclar com o dataset de temperatura
merged = pd.merge(weather, sivep_sub, on=['date','state'], how='left')

# Preencher NaNs de casos com zero (datas sem casos notificados)
merged['cases'] = merged['cases'].fillna(0).astype(int)

print('Shape merged:', merged.shape)
merged.head()


In [ ]:

# Estatísticas descritivas de temperatura por estado
summary_temp = weather.groupby('state')['temperature'].describe()
display(summary_temp)

# Plotar série temporal de temperatura para cada estado
plt.figure(figsize=(10,4))
for state in weather['state'].unique():
    subset = weather[weather['state']==state]
    plt.plot(subset['date'], subset['temperature'], label=state)
plt.title('Temperatura média diária (T2M) em 2019 por estado')
plt.xlabel('Data')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

# Calcular correlação entre temperatura e casos por estado
corrs = {}
for state in merged['state'].unique():
    df_state = merged[merged['state']==state]
    if len(df_state) > 0:
        corrs[state] = df_state['temperature'].corr(df_state['cases'])
print('Correlação temperatura x casos por estado:')
for st, corr in corrs.items():
    print(f"{st}: {corr:.3f}")

# Plotar gráficos de dispersão
for state in merged['state'].unique():
    df_state = merged[merged['state']==state]
    plt.figure(figsize=(6,4))
    plt.scatter(df_state['temperature'], df_state['cases'], s=10)
    plt.title(f'Casos de SRAG vs Temperatura (Estado: {state})')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Casos de SRAG')
    plt.tight_layout()
    plt.show()


## Observações

Os gráficos e a correlação calculada sugerem algumas relações potenciais entre temperatura e casos de SRAG em 2019 para os estados analisados:

- **São Paulo (SP)**: a correlação negativa observada indica que, em média, os casos de SRAG aumentaram quando as temperaturas caíram, o que pode refletir a sazonalidade de doenças respiratórias no inverno.
- **Rio de Janeiro (RJ)**: a correlação também é negativa, mas mais fraca. Há tendência similar de aumento de casos em períodos mais frios, porém com maior dispersão.
- **Amazonas (AM)**: a correlação próxima de zero sugere que, para este estado amazônico de clima tropical, a temperatura média diária pode não ter grande influência sobre a incidência de SRAG, e outros fatores (umidade, precipitação, condições socioambientais) devem ser considerados.

Estas análises são preliminares e servem de exemplo de como cruzar dados meteorológicos com notificações de doenças. Para conclusões mais robustas, seria necessário incorporar mais variáveis (umidade, poluição do ar, dados demográficos, etc.), ajustar por fatores confundidores e utilizar técnicas de inferência causal apropriadas.